#Setup Environment

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

%cd /gdrive/My Drive/TFG

!git pull https://github.com/ZhihaoDC/TFG

#Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx

# import matplotlib.pyplot as plt
# import matplotlib.colors as mcolors
# import seaborn as sns

# import statistics
# import math
# import itertools
# import re #regular expressions

# import plotly.express as px
# import plotly.figure_factory as ff